In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import pandas as pd
import numpy as np

2024-07-05 15:34:12.774198: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
## load data
x_train = pd.read_csv("../../../data/05/dii/x_train.csv", index_col=0, header=0)
y_train = pd.read_csv("../../../data/05/dii/y_train.csv", index_col = 0, header = 0 )
sw_train = pd.read_csv("../../../data/05/dii/sw_train.csv", index_col = 0, header = 0 )
x_test = pd.read_csv("../../../data/05/dii/x_test.csv", index_col=0, header=0)
y_test = pd.read_csv("../../../data/05/dii/y_test.csv", index_col = 0, header = 0 )
x_test = x_test[x_train.columns] # reordered columns to match (column ordered changed during taxahfe)

x_train = x_train.sort_index(axis = 0)
y_train = y_train.sort_index(axis = 0)
sw_train = sw_train.sort_index(axis = 0)
x_test = x_test.sort_index(axis = 0)
y_test = y_test.sort_index(axis = 0)

x_train = x_train.reset_index()
y_train = y_train.reset_index()
sw_train = sw_train.reset_index()
x_test = x_test.reset_index()
y_test = y_test.reset_index()

x_train = x_train.drop(columns='SEQN')
y_train = y_train.drop(columns='SEQN')
sw_train = sw_train.drop(columns='SEQN')
x_test = x_test.drop(columns='SEQN')
y_test = y_test.drop(columns='SEQN')

x_train = np.array(x_train)
y_train = np.array(y_train)
sample_weights = np.array(sw_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [3]:
keras.utils.set_random_seed(0)

In [4]:
acc = keras.metrics.Accuracy()

In [5]:
# baseline model
def model_builder(hp):
	# create model
    model = keras.Sequential()
    model.add(keras.layers.BatchNormalization(input_shape=(21,)))

	# Tune the number of units in the first Dense layer
    hp_units_1 = hp.Int('units_1', min_value=32, max_value=256, step=16)
    model.add(keras.layers.Dense(units=hp_units_1, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

	# Compile model
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), sample_weight_mode=sample_weights, metrics=['accuracy'])
    return model

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='tune',
                     project_name='binary_class_dii')

Reloading Tuner from tune/binary_class_dii/tuner0.json


In [7]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
lr_curve = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1,
    patience=5)

In [8]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early, lr_curve])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units_1')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 160 and the optimal learning rate for the optimizer
is 0.01.



In [9]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

acc_per_epoch = history.history['val_accuracy']
best_epoch = acc_per_epoch.index(max(acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
259/259 [==============================] - 1s 3ms/step - loss: 0.5703 - accuracy: 0.7117 - val_loss: 0.5270 - val_accuracy: 0.7323
Epoch 2/50
259/259 [==============================] - 1s 2ms/step - loss: 0.5402 - accuracy: 0.7276 - val_loss: 0.5021 - val_accuracy: 0.7502
Epoch 3/50
259/259 [==============================] - 0s 2ms/step - loss: 0.5318 - accuracy: 0.7326 - val_loss: 0.5026 - val_accuracy: 0.7498
Epoch 4/50
259/259 [==============================] - 1s 2ms/step - loss: 0.5298 - accuracy: 0.7349 - val_loss: 0.4855 - val_accuracy: 0.7565
Epoch 5/50
259/259 [==============================] - 1s 2ms/step - loss: 0.5236 - accuracy: 0.7410 - val_loss: 0.4892 - val_accuracy: 0.7609
Epoch 6/50
259/259 [==============================] - 1s 2ms/step - loss: 0.5253 - accuracy: 0.7429 - val_loss: 0.4795 - val_accuracy: 0.7633
Epoch 7/50
259/259 [==============================] - 1s 2ms/step - loss: 0.5223 - accuracy: 0.7387 - val_loss: 0.4848 - val_accuracy: 0.7541
Epoch 

In [10]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/37
259/259 [==============================] - 1s 2ms/step - loss: 0.5692 - accuracy: 0.7138 - val_loss: 0.5224 - val_accuracy: 0.7285
Epoch 2/37
259/259 [==============================] - 0s 1ms/step - loss: 0.5392 - accuracy: 0.7267 - val_loss: 0.5053 - val_accuracy: 0.7478
Epoch 3/37
259/259 [==============================] - 0s 2ms/step - loss: 0.5317 - accuracy: 0.7320 - val_loss: 0.5030 - val_accuracy: 0.7493
Epoch 4/37
259/259 [==============================] - 0s 1ms/step - loss: 0.5303 - accuracy: 0.7361 - val_loss: 0.4889 - val_accuracy: 0.7614
Epoch 5/37
259/259 [==============================] - 0s 2ms/step - loss: 0.5235 - accuracy: 0.7412 - val_loss: 0.4863 - val_accuracy: 0.7546
Epoch 6/37
259/259 [==============================] - 0s 2ms/step - loss: 0.5253 - accuracy: 0.7437 - val_loss: 0.4808 - val_accuracy: 0.7609
Epoch 7/37
259/259 [==============================] - 0s 2ms/step - loss: 0.5220 - accuracy: 0.7414 - val_loss: 0.4847 - val_accuracy: 0.7536
Epoch 

In [11]:
b_acc = keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)

In [12]:
pred = hypermodel(x_test)
b_acc.update_state(y_test, pred)
result = b_acc.result()
result.numpy()

0.7525174

In [13]:
# save model
hypermodel.save('model/binary_class_dii.keras')